# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import gkey

# Access maps with unique API key
gmaps.configure(api_key=gkey)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_df = pd.read_csv('cities.csv', index_col='City_ID')
city_df.head()


,country_code,city,lat,lng,Date,max_temp,humidity,wind_speed,cloudiness
City_ID,,,,,,,,,
0,au,albany,-38.850129,115.8613,1650983916,13.86,64,64,92
1,ru,saskylakh,73.943841,115.8613,1650983916,-20.08,100,100,76
2,ph,balabac,8.478664,115.8613,1650983916,29.18,71,71,100
3,au,karratha,-18.695084,115.8613,1650983917,27.69,63,63,100
4,my,kota kinabalu,6.017033,115.8613,1650983917,26.06,72,72,90


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [36]:
#Create a map using state centroid coordiantes to set markers 
locations = city_df[['lat','lng']]
w = city_df['humidity'].astype(float)

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=w,
                            dissipating=False, max_intensity=10,
                            point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [24]:
#set weather criteria parameter
max_cloud = 50
max_humidity = 20

#store into new dataframe
hotel_df = city_df.loc[(city_df['cloudiness'] <= max_cloud) & (city_df['humidity'] <= max_humidity)].reset_index()

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [30]:
hotel_df['Hotel Name'] = ''
hotel_df.head()

,City_ID,country_code,city,lat,lng,Date,max_temp,humidity,wind_speed,cloudiness,hotel_Name,hotel Name,Hotel Name
0,5,cn,renqiu,38.708675,115.8613,1650983917,15.64,12,12,1,,,
1,18,cn,hengshui,37.593594,115.8613,1650983919,17.24,11,11,18,,,


In [33]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country_code}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations_1 = hotel_df[["lat", "lng"]]

In [34]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations_1)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))

['\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><dd>renqiu</dd>\n<dt>Country</dt><dd>cn</dd>\n</dl>\n', '\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><dd>hengshui</dd>\n<dt>Country</dt><dd>cn</dd>\n</dl>\n']
